# Importando bibliotecas necessárias

In [2]:
import pandas as pd
import requests
import zipfile
import io

# Script para puxar base de dados 

In [3]:
url = 'https://www.compras.rj.gov.br/siga/imagens/CONTRATOS.zip'
response = requests.get(url)

if response.status_code == 200:
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    
    dataframes = {}
    
    for file in zip_file.filelist:
        if file.filename.lower().endswith('.csv'):
            with zip_file.open(file) as csv_file:
                dataframe = pd.read_csv(csv_file, encoding='latin1', sep=';')
                dataframes[file.filename] = dataframe

else:
    print("Falha ao baixar arquivo.")

C:\Users\gabri\AppData\Local\Temp\ipykernel_14356\1324880437.py:12: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(csv_file, encoding='latin1', sep=';')


# Limpeza e Tratamento dos dados

In [4]:
for key, dataframe in dataframes.items():
    print('-'*80)
    print(f'Arquivo: {key}')
    print('-'*80)
    print(dataframe.isnull().sum())
     # Observei alguns valores nulos, vou analisar melhor na próxima célula
    dataframe_contratos = dataframes['CONTRATOS.CSV']
    dataframe_itens = dataframes['ITENS_CONTRATOS.CSV']

--------------------------------------------------------------------------------
Arquivo: CONTRATOS.CSV
--------------------------------------------------------------------------------
Contratação                                                      0
Status Contratação                                               0
Data Contratação                                                 0
Unidade                                                          0
Processo                                                       528
Objeto                                                         530
Tipo de Aquisição                                              528
Critério de Julgamento                                           0
Data Início Vigência                                         24456
Data Fim Vigência                                            24574
Fornecedor                                                       0
CPF/CNPJ                                                         0
Valor Total

In [5]:
display(dataframe_itens.loc[dataframe_itens['VL. Unit.Original'].isnull()])

# Para concertar os valores vazios do dataframes_itens precisamos mesclar com dataframe_contratos e pegar o valor total

,Contratação,ID Item,Item,Qtde Original,VL. Unit.Original,Total Aditivada/Suprimida,VL. Unit.Aditivado/Suprimido,data_extracao
40014,2013000026,69007,CARTUCHO TINTA / TINTEIRO IMPRESSORA - TIPO: O...,10,NaN,0,0,27/11/2023
40015,2013000026,69247,CARTUCHO TINTA / TINTEIRO IMPRESSORA - TIPO: O...,10,NaN,0,0,27/11/2023
40016,2013000026,97716,CARTUCHO TINTA / TINTEIRO IMPRESSORA - TIPO: O...,15,NaN,0,0,27/11/2023


In [6]:
# Observei que as colunas Processo, Objeto e Tipo de Aquisição está com valores próximos nulos, vamos analisar melhor abaixo:

print(len(dataframe_contratos[dataframe_contratos['Processo'].isnull()]))
print(len(dataframe_contratos[dataframe_contratos['Tipo de Aquisição'].isnull()]))
print(len(dataframe_contratos[(dataframe_contratos['Tipo de Aquisição'].isnull()) & (dataframe_contratos['Processo'].isnull())]))
print(len(dataframe_contratos[(dataframe_contratos['Tipo de Aquisição'].isnull()) & (dataframe_contratos['Objeto'].isnull())]))

# Com isso conclui que onde Processo é nulo, tipo de aquisição e objeto também é.
# Porém objeto tem 2 a mais, vamos verificar esses dois a mais na próxima célula


528
528
528
528


In [7]:
diferenca = dataframe_contratos[dataframe_contratos['Objeto'].isnull() != (dataframe_contratos['Processo'].isnull())]
print(len(diferenca))
display(diferenca)

# Os dois valores nulos na coluna objeto que estão com valores vazios representam status Em Aberto e com grande
# relevância monetária não sendo indicado a exclusão dessas linhas. Com isso precisamos entrar em contato com o setor responsável
# verificar se a coluna objeto é de relevância para a análise ou se precisa ser preenchido.

2


,Contratação,Status Contratação,Data Contratação,Unidade,Processo,Objeto,Tipo de Aquisição,Critério de Julgamento,Data Início Vigência,Data Fim Vigência,Fornecedor,CPF/CNPJ,Valor Total Contrato/Valor Estimado para Contratação (R$),Valor Total Empenhado (R$),Valor Total Liquidado (R$),Valor Total Pago (R$),Data Public DEORJ,data_extracao
97095,2023009497,Em Aberto,14/11/2023,FUNESPOM - FUNDO ESP. POLÍCIA MILITAR RJ,SEI-350207/000208B/2023,NaN,Solicitação de Compra,Menor Preço,07/11/2023,06/11/2024,LEMAN MEDICAMENTOS E CIA LTDA,40.600.760/0001-54,"10339,8",NaN,NaN,NaN,07/11/2023,27/11/2023
99276,2023006598,Em Aberto,22/08/2023,CECIERJ - FUND CENTRO CIÊN EDUC SUP DISTÂN DO ...,SEI-260004/1891/2020,NaN,Dispensa - Especial,Menor Preço,14/05/2021,13/05/2026,MARIA CRISTINA DOS SANTOS,579.771.867-72,"552616,512","62.278,02",NaN,NaN,NaN,27/11/2023


In [8]:
# Vou analisar agora Data Início Vigência e Data Fim Vigência
# Vou observar se sempre que Data Início Vigência for vazio o Fim também é

print((len(dataframe_contratos[dataframe_contratos['Data Início Vigência'].isnull()])))
print((len(dataframe_contratos[dataframe_contratos['Data Fim Vigência'].isnull()])))
print(len(dataframe_contratos[(dataframe_contratos['Data Fim Vigência'].isnull()) & (dataframe_contratos['Data Início Vigência'].isnull())]))

# Foi observado que na maioria das vezes que Data Início está vazia, Data Fim também está.
# Porém vou seguir a lógica de afirmar que sempre que contém Data Início mas não contém Data Final há alguma correlação com Status
# Se houver Data Final mas não houver Data Início, é algum erro de 'inputação' de dados
# Vou observar os dois casos na próxima célula


24456
24574
24439


In [9]:
# Observando onde há data fim mas não há data início

print(len(dataframe_contratos[(dataframe_contratos['Data Fim Vigência'].notnull()) & (dataframe_contratos['Data Início Vigência'].isnull())]))

# Conforme esperado, apenas 17 são valores que há Fim mas não há início, sendo possívelmente um erro de inputação de dados.
# Vou observar ao contrário, em outra célula

17


In [10]:
# Observando onde há data início mas não há data fim

print(len(dataframe_contratos[(dataframe_contratos['Data Fim Vigência'].isnull()) & (dataframe_contratos['Data Início Vigência'].notnull())]))

# Obeservando a correlação com o status agora.


display(dataframe_contratos[(dataframe_contratos['Data Fim Vigência'].isnull()) & (dataframe_contratos['Data Início Vigência'].notnull())])

135


,Contratação,Status Contratação,Data Contratação,Unidade,Processo,Objeto,Tipo de Aquisição,Critério de Julgamento,Data Início Vigência,Data Fim Vigência,Fornecedor,CPF/CNPJ,Valor Total Contrato/Valor Estimado para Contratação (R$),Valor Total Empenhado (R$),Valor Total Liquidado (R$),Valor Total Pago (R$),Data Public DEORJ,data_extracao
54,2012001100,Encerrado,16/03/2012,CODIN - COMP DE DESENV INDUSTRIAL DO ESTADO DO RJ,E-11/30.001/2010,Contratação de escritório de advocacia para de...,Renovação de Contrato,Menor Preço,01/03/2012,NaN,JOSE RIBAMAR GARCIA ADVOGADOS ASSOCIADOS,03.486.963/0001-35,"28362,116",NaN,NaN,NaN,NaN,27/11/2023
274,2011000063,Encerrado,25/07/2011,DRM - DEPART RECURSOS MINERAIS ESTADO RJ,E-11/40.296/2011,Participação de Coordenadores no ``II Encontro...,Pequenas Compras,Menor Preço,29/07/2011,NaN,IMAGEM GEOSISTEMAS E COMÉRCIO LTDA,67.393.181/0001-34,"956,8",NaN,NaN,NaN,NaN,27/11/2023
297,2011000020,Encerrado,17/06/2011,"SEELJE - SEC DE EST ESPORTE, LAZER E JUVENTUDE",E-30/000.553/2011,Aquisição de livros ``Maracanã 60 anos``,Inexigibilidade,Menor Preço,20/06/2011,NaN,DIVULGADORA BUENO & BUENO LTDA,03.477.925/0001-16,22000,NaN,NaN,NaN,NaN,27/11/2023
302,2011000026,Encerrado,29/06/2011,"SEELJE - SEC DE EST ESPORTE, LAZER E JUVENTUDE",E-30/000.448/2011,Prestação de Serviços de Locação de ônibus,Pregão Eletrônico,Menor Preço,22/06/2011,NaN,VENUS TURISTICA LTDA,29.468.329/0001-63,63994,NaN,NaN,NaN,NaN,27/11/2023
304,2011000028,Encerrado,30/06/2011,"SEELJE - SEC DE EST ESPORTE, LAZER E JUVENTUDE",E-30/000.448/2011,Prestação de Serviços de Locação de ônibus,Pregão Eletrônico,Menor Preço,22/06/2011,NaN,TRANSPORTE E TURISMO REAL BRASIL LTDA,40.160.558/0001-59,121516,NaN,NaN,NaN,NaN,27/11/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80408,2021006373,Em Aberto,18/11/2021,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,SEI-04/172/000033A/2019,Trata-se da prorrogação do Contrato n° 10/2019...,Renovação de Contrato,Menor Preço,13/12/2021,NaN,TRIVALE INSTITUICAO DE PAGAMENTO LTDA,00.604.122/0001-97,"801354,80259","702.665,27","57.464,45","57.464,45",NaN,27/11/2023
89629,2022009476,Em Aberto,23/11/2022,FAF - Fundo Especial de Adm Fazendária,SEI-040063/000015A/2020SEI-040063/000015B/2020,Tarata-se da contratação de serviços de proces...,Pregão Eletrônico,Menor Preço,28/11/2022,NaN,VS DATA COMERCIAL INFORMATICA LTDA,07.268.152/0004-61,"86980808,33","9.891.890,58","71.641,77","70.567,14",NaN,27/11/2023
89846,2022009534,Em Aberto,25/11/2022,SEDSODH - SEC ESTADO DESEN SOCIAL E DE DIR HUM,SEI-150001/014707/2021,cumprimento do acordo,Dispensa - Especial,Menor Preço,01/06/2022,NaN,OI S/A,76.535.764/0001-43,"1956187,86",NaN,NaN,NaN,NaN,27/11/2023
95079,2023005714,Em Aberto,25/07/2023,EMOP - EMPRESA DE OBRAS PÚBLICAS DO ESTADO DO RJ,SEI-170002/002537/2022,PRESTAÇÃO DE SERVIÇOS DE MANUTENÇÃO PREVENTIVA...,Inexigibilidade,Menor Preço,19/07/2023,NaN,ELEVADORES ALPHA LTDA,33.274.994/0001-93,"665167,75","665.167,75","594.052,26","594.052,26",NaN,27/11/2023


In [11]:
# Parece que todos estão Cancelado, Em Aberto ou Encerrado, vou confirmar

datas = dataframe_contratos[(dataframe_contratos['Data Fim Vigência'].isnull()) & (dataframe_contratos['Data Início Vigência'].notnull())]
ativos = datas[(datas['Status Contratação'] != 'Em Aberto') & (datas['Status Contratação'] != 'Cancelado') & (datas['Status Contratação'] != 'Encerrado')]

# Observei que mesmo diferente há contratos Ativos que não há data de fim porém há de início, concluindo que também
# é um erro de inputação de dados.


# Correlação Contratos x Itens Contratos

In [12]:
dataframe_contratos = dataframe_contratos.drop('data_extracao', axis=1)
dataframe_correlacao = dataframe_contratos.merge(dataframe_itens, on = 'Contratação')
display(dataframe_correlacao)

,Contratação,Status Contratação,Data Contratação,Unidade,Processo,Objeto,Tipo de Aquisição,Critério de Julgamento,Data Início Vigência,Data Fim Vigência,...,Valor Total Liquidado (R$),Valor Total Pago (R$),Data Public DEORJ,ID Item,Item,Qtde Original,VL. Unit.Original,Total Aditivada/Suprimida,VL. Unit.Aditivado/Suprimido,data_extracao
0,2011000033,Encerrado,01/07/2011,SEPLAG - SEC DE EST DE PLANEJAMENTO E GESTÃO (...,E-01/0013/2011,AQUISIÇÃO DE MATERIAL DE EXPEDIENTE ATRAVÉS DO...,Solicitação de Compra,Menor Preço,18/04/2011,08/07/2011,...,NaN,NaN,NaN,32404,ENVELOPE CORRESPONDENCIA PADRAO - MODELO: OFIC...,2500,",03",0,0,27/11/2023
1,2011000034,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04-011.578/2010,Contratação de empresa especializada na presta...,Pregão Eletrônico,Menor Preço,23/08/2011,20/02/2012,...,NaN,NaN,NaN,63031,MODERNIZACAO DE HARDWARE E SOFTWARE - DESCRICA...,1,368962,0,0,27/11/2023
2,2011000035,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,01/09/2011,15/09/2011,...,NaN,NaN,NaN,65691,"CORTINA - MATERIAL: TECIDO, MODELO: ROLO, DIME...","2065,5","146,101",0,0,27/11/2023
3,2011000035,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,01/09/2011,15/09/2011,...,NaN,NaN,NaN,65718,"CORTINA BLACK OUT - MATERIAL: TECIDO, FIXACAO:...",128,"84,1284",0,0,27/11/2023
4,2011000036,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/008 . 249/2010,Contratação de empresa especializada para pres...,Pregão Eletrônico,Menor Preço,12/06/2011,11/07/2011,...,NaN,NaN,NaN,64206,CARIMBO - DESCRIÇÃO: CONFECCAO DE CARIMBO AUTO...,60,14,0,0,27/11/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279592,2023007084,Em Aberto,05/09/2023,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/005557/2022,PROJETO EXECUTIVO E OBRAS DE URBANIZAÇÃO DO LA...,Concorrência,Menor Preço,04/08/2023,04/07/2024,...,NaN,NaN,04/08/2023,173516,CONTRATACAO DE SERVICOS DE URBANIZACAO - DESCR...,1,"4083067,07",0,0,27/11/2023
279593,2023007086,Em Aberto,05/09/2023,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/007283/2022,OBRAS DE RECUPERAÇÃO DA BARRAGEM DE GERICINÓ N...,Concorrência,Menor Preço,17/08/2023,17/06/2024,...,NaN,NaN,17/08/2023,168278,SERVICOS COMPLEMENTARES - DESCRICAO: CONTRATAC...,1,13851000,0,0,27/11/2023
279594,2023007087,Em Aberto,05/09/2023,FSERJ - FUNDAÇÃO SAÚDE DO EST. DO RIO DE JANEIRO,SEI-080007/011322/2023,Contratação de empresa especialiada para prest...,Dispensa - Especial,Menor Preço,NaN,NaN,...,NaN,NaN,NaN,151817,SERVICOS DE APOIO AS ATIVIDADES ADMINISTRATIVA...,1,"12476040,91",0,0,27/11/2023
279595,2023007088,Em Aberto,05/09/2023,FUNARJ - FUND ANITA MANTUANO DE ARTES DO EST D...,SEI-180002/001295/2023,O objeto do presente Termo de Referência é a C...,Inexigibilidade,Menor Preço,NaN,NaN,...,NaN,NaN,NaN,46390,SERVICOS DE ORGANIZACAO DE EVENTOS - DESCRIÇÃO...,1,5000,0,0,27/11/2023


In [13]:
# Excluindo algumas linhas iguais

dataframe_correlacao.drop_duplicates(ignore_index=True)

,Contratação,Status Contratação,Data Contratação,Unidade,Processo,Objeto,Tipo de Aquisição,Critério de Julgamento,Data Início Vigência,Data Fim Vigência,...,Valor Total Liquidado (R$),Valor Total Pago (R$),Data Public DEORJ,ID Item,Item,Qtde Original,VL. Unit.Original,Total Aditivada/Suprimida,VL. Unit.Aditivado/Suprimido,data_extracao
0,2011000033,Encerrado,01/07/2011,SEPLAG - SEC DE EST DE PLANEJAMENTO E GESTÃO (...,E-01/0013/2011,AQUISIÇÃO DE MATERIAL DE EXPEDIENTE ATRAVÉS DO...,Solicitação de Compra,Menor Preço,18/04/2011,08/07/2011,...,NaN,NaN,NaN,32404,ENVELOPE CORRESPONDENCIA PADRAO - MODELO: OFIC...,2500,",03",0,0,27/11/2023
1,2011000034,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04-011.578/2010,Contratação de empresa especializada na presta...,Pregão Eletrônico,Menor Preço,23/08/2011,20/02/2012,...,NaN,NaN,NaN,63031,MODERNIZACAO DE HARDWARE E SOFTWARE - DESCRICA...,1,368962,0,0,27/11/2023
2,2011000035,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,01/09/2011,15/09/2011,...,NaN,NaN,NaN,65691,"CORTINA - MATERIAL: TECIDO, MODELO: ROLO, DIME...","2065,5","146,101",0,0,27/11/2023
3,2011000035,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,01/09/2011,15/09/2011,...,NaN,NaN,NaN,65718,"CORTINA BLACK OUT - MATERIAL: TECIDO, FIXACAO:...",128,"84,1284",0,0,27/11/2023
4,2011000036,Encerrado,01/07/2011,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/008 . 249/2010,Contratação de empresa especializada para pres...,Pregão Eletrônico,Menor Preço,12/06/2011,11/07/2011,...,NaN,NaN,NaN,64206,CARIMBO - DESCRIÇÃO: CONFECCAO DE CARIMBO AUTO...,60,14,0,0,27/11/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278947,2023007084,Em Aberto,05/09/2023,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/005557/2022,PROJETO EXECUTIVO E OBRAS DE URBANIZAÇÃO DO LA...,Concorrência,Menor Preço,04/08/2023,04/07/2024,...,NaN,NaN,04/08/2023,173516,CONTRATACAO DE SERVICOS DE URBANIZACAO - DESCR...,1,"4083067,07",0,0,27/11/2023
278948,2023007086,Em Aberto,05/09/2023,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/007283/2022,OBRAS DE RECUPERAÇÃO DA BARRAGEM DE GERICINÓ N...,Concorrência,Menor Preço,17/08/2023,17/06/2024,...,NaN,NaN,17/08/2023,168278,SERVICOS COMPLEMENTARES - DESCRICAO: CONTRATAC...,1,13851000,0,0,27/11/2023
278949,2023007087,Em Aberto,05/09/2023,FSERJ - FUNDAÇÃO SAÚDE DO EST. DO RIO DE JANEIRO,SEI-080007/011322/2023,Contratação de empresa especialiada para prest...,Dispensa - Especial,Menor Preço,NaN,NaN,...,NaN,NaN,NaN,151817,SERVICOS DE APOIO AS ATIVIDADES ADMINISTRATIVA...,1,"12476040,91",0,0,27/11/2023
278950,2023007088,Em Aberto,05/09/2023,FUNARJ - FUND ANITA MANTUANO DE ARTES DO EST D...,SEI-180002/001295/2023,O objeto do presente Termo de Referência é a C...,Inexigibilidade,Menor Preço,NaN,NaN,...,NaN,NaN,NaN,46390,SERVICOS DE ORGANIZACAO DE EVENTOS - DESCRIÇÃO...,1,5000,0,0,27/11/2023


In [14]:
data_string = '18/11/2023'
data_formatada = pd.to_datetime(data_string, format='%d/%m/%Y')
print(data_formatada)

2023-11-18 00:00:00


In [20]:
# convertendo o tipo de dado para data

dataframe_correlacao['Data Contratação'] = pd.to_datetime(dataframe_correlacao['Data Contratação'], format='%d/%m/%Y')
dataframe_correlacao['Data Início Vigência'] = pd.to_datetime(dataframe_correlacao['Data Início Vigência'], format='%d/%m/%Y')
dataframe_correlacao['data_extracao'] = pd.to_datetime(dataframe_correlacao['data_extracao'], format='%d/%m/%Y')

In [16]:
dataframe_correlacao.drop('Data Public DEORJ', axis = 1, inplace=True)

In [17]:
display(dataframe_correlacao)

,Contratação,Status Contratação,Data Contratação,Unidade,Processo,Objeto,Tipo de Aquisição,Critério de Julgamento,Data Início Vigência,Data Fim Vigência,...,Valor Total Empenhado (R$),Valor Total Liquidado (R$),Valor Total Pago (R$),ID Item,Item,Qtde Original,VL. Unit.Original,Total Aditivada/Suprimida,VL. Unit.Aditivado/Suprimido,data_extracao
0,2011000033,Encerrado,2011-07-01,SEPLAG - SEC DE EST DE PLANEJAMENTO E GESTÃO (...,E-01/0013/2011,AQUISIÇÃO DE MATERIAL DE EXPEDIENTE ATRAVÉS DO...,Solicitação de Compra,Menor Preço,2011-04-18,08/07/2011,...,NaN,NaN,NaN,32404,ENVELOPE CORRESPONDENCIA PADRAO - MODELO: OFIC...,2500,",03",0,0,2023-11-27
1,2011000034,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04-011.578/2010,Contratação de empresa especializada na presta...,Pregão Eletrônico,Menor Preço,2011-08-23,20/02/2012,...,NaN,NaN,NaN,63031,MODERNIZACAO DE HARDWARE E SOFTWARE - DESCRICA...,1,368962,0,0,2023-11-27
2,2011000035,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,2011-09-01,15/09/2011,...,NaN,NaN,NaN,65691,"CORTINA - MATERIAL: TECIDO, MODELO: ROLO, DIME...","2065,5","146,101",0,0,2023-11-27
3,2011000035,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,2011-09-01,15/09/2011,...,NaN,NaN,NaN,65718,"CORTINA BLACK OUT - MATERIAL: TECIDO, FIXACAO:...",128,"84,1284",0,0,2023-11-27
4,2011000036,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/008 . 249/2010,Contratação de empresa especializada para pres...,Pregão Eletrônico,Menor Preço,2011-06-12,11/07/2011,...,NaN,NaN,NaN,64206,CARIMBO - DESCRIÇÃO: CONFECCAO DE CARIMBO AUTO...,60,14,0,0,2023-11-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279592,2023007084,Em Aberto,2023-09-05,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/005557/2022,PROJETO EXECUTIVO E OBRAS DE URBANIZAÇÃO DO LA...,Concorrência,Menor Preço,2023-08-04,04/07/2024,...,"2.674.838,04",NaN,NaN,173516,CONTRATACAO DE SERVICOS DE URBANIZACAO - DESCR...,1,"4083067,07",0,0,2023-11-27
279593,2023007086,Em Aberto,2023-09-05,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/007283/2022,OBRAS DE RECUPERAÇÃO DA BARRAGEM DE GERICINÓ N...,Concorrência,Menor Preço,2023-08-17,17/06/2024,...,"972.509,00",NaN,NaN,168278,SERVICOS COMPLEMENTARES - DESCRICAO: CONTRATAC...,1,13851000,0,0,2023-11-27
279594,2023007087,Em Aberto,2023-09-05,FSERJ - FUNDAÇÃO SAÚDE DO EST. DO RIO DE JANEIRO,SEI-080007/011322/2023,Contratação de empresa especialiada para prest...,Dispensa - Especial,Menor Preço,NaT,NaN,...,"1.736.260,00",NaN,NaN,151817,SERVICOS DE APOIO AS ATIVIDADES ADMINISTRATIVA...,1,"12476040,91",0,0,2023-11-27
279595,2023007088,Em Aberto,2023-09-05,FUNARJ - FUND ANITA MANTUANO DE ARTES DO EST D...,SEI-180002/001295/2023,O objeto do presente Termo de Referência é a C...,Inexigibilidade,Menor Preço,NaT,NaN,...,"5.000,00",NaN,NaN,46390,SERVICOS DE ORGANIZACAO DE EVENTOS - DESCRIÇÃO...,1,5000,0,0,2023-11-27


In [18]:
dataframe_correlacao["Data Início Vigência"] = dataframe_correlacao["Data Início Vigência"].fillna(method="ffill")
dataframe_correlacao["Data Fim Vigência"] = dataframe_correlacao["Data Fim Vigência"].fillna(method="ffill")
display(dataframe_correlacao)

,Contratação,Status Contratação,Data Contratação,Unidade,Processo,Objeto,Tipo de Aquisição,Critério de Julgamento,Data Início Vigência,Data Fim Vigência,...,Valor Total Empenhado (R$),Valor Total Liquidado (R$),Valor Total Pago (R$),ID Item,Item,Qtde Original,VL. Unit.Original,Total Aditivada/Suprimida,VL. Unit.Aditivado/Suprimido,data_extracao
0,2011000033,Encerrado,2011-07-01,SEPLAG - SEC DE EST DE PLANEJAMENTO E GESTÃO (...,E-01/0013/2011,AQUISIÇÃO DE MATERIAL DE EXPEDIENTE ATRAVÉS DO...,Solicitação de Compra,Menor Preço,2011-04-18,08/07/2011,...,NaN,NaN,NaN,32404,ENVELOPE CORRESPONDENCIA PADRAO - MODELO: OFIC...,2500,",03",0,0,2023-11-27
1,2011000034,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04-011.578/2010,Contratação de empresa especializada na presta...,Pregão Eletrônico,Menor Preço,2011-08-23,20/02/2012,...,NaN,NaN,NaN,63031,MODERNIZACAO DE HARDWARE E SOFTWARE - DESCRICA...,1,368962,0,0,2023-11-27
2,2011000035,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,2011-09-01,15/09/2011,...,NaN,NaN,NaN,65691,"CORTINA - MATERIAL: TECIDO, MODELO: ROLO, DIME...","2065,5","146,101",0,0,2023-11-27
3,2011000035,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,2011-09-01,15/09/2011,...,NaN,NaN,NaN,65718,"CORTINA BLACK OUT - MATERIAL: TECIDO, FIXACAO:...",128,"84,1284",0,0,2023-11-27
4,2011000036,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/008 . 249/2010,Contratação de empresa especializada para pres...,Pregão Eletrônico,Menor Preço,2011-06-12,11/07/2011,...,NaN,NaN,NaN,64206,CARIMBO - DESCRIÇÃO: CONFECCAO DE CARIMBO AUTO...,60,14,0,0,2023-11-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279592,2023007084,Em Aberto,2023-09-05,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/005557/2022,PROJETO EXECUTIVO E OBRAS DE URBANIZAÇÃO DO LA...,Concorrência,Menor Preço,2023-08-04,04/07/2024,...,"2.674.838,04",NaN,NaN,173516,CONTRATACAO DE SERVICOS DE URBANIZACAO - DESCR...,1,"4083067,07",0,0,2023-11-27
279593,2023007086,Em Aberto,2023-09-05,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/007283/2022,OBRAS DE RECUPERAÇÃO DA BARRAGEM DE GERICINÓ N...,Concorrência,Menor Preço,2023-08-17,17/06/2024,...,"972.509,00",NaN,NaN,168278,SERVICOS COMPLEMENTARES - DESCRICAO: CONTRATAC...,1,13851000,0,0,2023-11-27
279594,2023007087,Em Aberto,2023-09-05,FSERJ - FUNDAÇÃO SAÚDE DO EST. DO RIO DE JANEIRO,SEI-080007/011322/2023,Contratação de empresa especialiada para prest...,Dispensa - Especial,Menor Preço,2023-08-17,17/06/2024,...,"1.736.260,00",NaN,NaN,151817,SERVICOS DE APOIO AS ATIVIDADES ADMINISTRATIVA...,1,"12476040,91",0,0,2023-11-27
279595,2023007088,Em Aberto,2023-09-05,FUNARJ - FUND ANITA MANTUANO DE ARTES DO EST D...,SEI-180002/001295/2023,O objeto do presente Termo de Referência é a C...,Inexigibilidade,Menor Preço,2023-08-17,17/06/2024,...,"5.000,00",NaN,NaN,46390,SERVICOS DE ORGANIZACAO DE EVENTOS - DESCRIÇÃO...,1,5000,0,0,2023-11-27


In [19]:
dataframe_correlacao.to_excel('contratos_com_itens.xlsx', index=False)

In [21]:
display(dataframe_correlacao)

,Contratação,Status Contratação,Data Contratação,Unidade,Processo,Objeto,Tipo de Aquisição,Critério de Julgamento,Data Início Vigência,Data Fim Vigência,...,Valor Total Empenhado (R$),Valor Total Liquidado (R$),Valor Total Pago (R$),ID Item,Item,Qtde Original,VL. Unit.Original,Total Aditivada/Suprimida,VL. Unit.Aditivado/Suprimido,data_extracao
0,2011000033,Encerrado,2011-07-01,SEPLAG - SEC DE EST DE PLANEJAMENTO E GESTÃO (...,E-01/0013/2011,AQUISIÇÃO DE MATERIAL DE EXPEDIENTE ATRAVÉS DO...,Solicitação de Compra,Menor Preço,2011-04-18,08/07/2011,...,NaN,NaN,NaN,32404,ENVELOPE CORRESPONDENCIA PADRAO - MODELO: OFIC...,2500,",03",0,0,2023-11-27
1,2011000034,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04-011.578/2010,Contratação de empresa especializada na presta...,Pregão Eletrônico,Menor Preço,2011-08-23,20/02/2012,...,NaN,NaN,NaN,63031,MODERNIZACAO DE HARDWARE E SOFTWARE - DESCRICA...,1,368962,0,0,2023-11-27
2,2011000035,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,2011-09-01,15/09/2011,...,NaN,NaN,NaN,65691,"CORTINA - MATERIAL: TECIDO, MODELO: ROLO, DIME...","2065,5","146,101",0,0,2023-11-27
3,2011000035,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/012984/2010,Trata-se de Aquisição e montagem com entrega p...,Pregão Eletrônico,Menor Preço,2011-09-01,15/09/2011,...,NaN,NaN,NaN,65718,"CORTINA BLACK OUT - MATERIAL: TECIDO, FIXACAO:...",128,"84,1284",0,0,2023-11-27
4,2011000036,Encerrado,2011-07-01,SEFAZ - SECRETARIA DE ESTADO DE FAZENDA,E04/008 . 249/2010,Contratação de empresa especializada para pres...,Pregão Eletrônico,Menor Preço,2011-06-12,11/07/2011,...,NaN,NaN,NaN,64206,CARIMBO - DESCRIÇÃO: CONFECCAO DE CARIMBO AUTO...,60,14,0,0,2023-11-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279592,2023007084,Em Aberto,2023-09-05,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/005557/2022,PROJETO EXECUTIVO E OBRAS DE URBANIZAÇÃO DO LA...,Concorrência,Menor Preço,2023-08-04,04/07/2024,...,"2.674.838,04",NaN,NaN,173516,CONTRATACAO DE SERVICOS DE URBANIZACAO - DESCR...,1,"4083067,07",0,0,2023-11-27
279593,2023007086,Em Aberto,2023-09-05,INEA - INSTITUTO ESTADUAL DO AMBIENTE,SEI-070002/007283/2022,OBRAS DE RECUPERAÇÃO DA BARRAGEM DE GERICINÓ N...,Concorrência,Menor Preço,2023-08-17,17/06/2024,...,"972.509,00",NaN,NaN,168278,SERVICOS COMPLEMENTARES - DESCRICAO: CONTRATAC...,1,13851000,0,0,2023-11-27
279594,2023007087,Em Aberto,2023-09-05,FSERJ - FUNDAÇÃO SAÚDE DO EST. DO RIO DE JANEIRO,SEI-080007/011322/2023,Contratação de empresa especialiada para prest...,Dispensa - Especial,Menor Preço,2023-08-17,17/06/2024,...,"1.736.260,00",NaN,NaN,151817,SERVICOS DE APOIO AS ATIVIDADES ADMINISTRATIVA...,1,"12476040,91",0,0,2023-11-27
279595,2023007088,Em Aberto,2023-09-05,FUNARJ - FUND ANITA MANTUANO DE ARTES DO EST D...,SEI-180002/001295/2023,O objeto do presente Termo de Referência é a C...,Inexigibilidade,Menor Preço,2023-08-17,17/06/2024,...,"5.000,00",NaN,NaN,46390,SERVICOS DE ORGANIZACAO DE EVENTOS - DESCRIÇÃO...,1,5000,0,0,2023-11-27
